In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from opentisim import agribulk_objects
from opentisim import agribulk_defaults
from opentisim import agribulk_system

# 0. Prepare inputs

In [2]:
startyear = 2018
lifecycle = 10

## 0.1 Generate demand forecast scenario

In [3]:
years = list(range(startyear, startyear+lifecycle))

demand = []
for year in years:
    if year < 2023:
        demand.append(750_000)
    else:
        demand.append(1_600_000)

scenario_data={'year': years, 'volume': demand}

# instantiate a commodity objects
# maize
agribulk_defaults.maize_data['handysize_perc'] = 0
agribulk_defaults.maize_data['handymax_perc'] = 100
agribulk_defaults.maize_data['panamax_perc'] = 0
agribulk_defaults.maize_data['handling_fee'] = 9.8 # example Wijnand
agribulk_defaults.maize_data['historic_data'] = []
maize = agribulk_objects.Commodity(**agribulk_defaults.maize_data)
maize.scenario_data = pd.DataFrame(data=scenario_data)

# wheat
agribulk_defaults.wheat_data['historic_data'] = []
agribulk_defaults.wheat_data['scenario_data'] = []
wheat = agribulk_objects.Commodity(**agribulk_defaults.wheat_data)

# soybeans
agribulk_defaults.soybean_data['historic_data'] = []
agribulk_defaults.soybean_data['scenario_data'] = []
soybeans = agribulk_objects.Commodity(**agribulk_defaults.soybean_data)

# combine
demand = [maize, wheat, soybeans]

# plot scenarios 
if False:
    maize.plot_demand()
    wheat.plot_demand()
    soybeans.plot_demand()

## 0.2 Generate vessels

In [4]:
# instantiate vessels
handysize = agribulk_objects.Vessel(**agribulk_defaults.handysize_data)
agribulk_defaults.handymax_data['call_size'] = 50000
handymax = agribulk_objects.Vessel(**agribulk_defaults.handymax_data)
panamax = agribulk_objects.Vessel(**agribulk_defaults.panamax_data)


vessels = [handysize, handymax, panamax] 

# 1. Instatiate terminal system

In [5]:
Terminal = agribulk_system.System(startyear = startyear, 
                         lifecycle = lifecycle, 
                         elements = demand + vessels, 
                         operational_hours = 5840,   # example Wijnand
                         debug=True,
                         crane_type_defaults=agribulk_defaults.mobile_crane_data, 
                         storage_type_defaults=agribulk_defaults.silo_data)

# 2. Start simulation
Develop terminal given the throughput scenario and investment triggers implemented

In [6]:
Terminal.simulate()


### Simulate year: 2018 ############################
--- Cargo volume and vessel calls for 2018 ---------
  Total cargo volume: 750000
  Total vessel calls: 15
     Handysize calls: 0
     Handymax calls: 15
     Panamax calls: 0
----------------------------------------------------

--- Status terminal @ start of year ----------------
     Berth occupancy online (@ start of year): inf (trigger level: 0.40)
     Berth occupancy planned (@ start of year): inf (trigger level: 0.40)
     Planned waiting time service time factor (@ start of year): inf (trigger level: 0.30)

--- Start investment analysis ----------------------

$$$ Check berth elements (coupled with berth occupancy) ---------------
  *** add Berth to elements
     Berth occupancy planned (after adding berth): inf (trigger level: 0.40)
     Planned waiting time service time factor : 4.36 (trigger level: 0.30)
  *** add Quay to elements
     Berth occupancy planned (after adding berth): inf (trigger level: 0.40)
     Planned 

AttributeError: 'list' object has no attribute 'loc'

NameError: name 'commodity' is not defined

## Report all elements

In [ ]:
# if False: 
for element in Terminal.elements:
    print("")
    print(element.name)
    print("")
    print(element.__dict__)

In [ ]:
Terminal.terminal_elements_plot()

In [ ]:
Terminal.terminal_capacity_plot()

In [ ]:
# add cash flow information for each of the Terminal elements
cash_flows, cash_flows_WACC_real = Terminal.add_cashflow_elements()

In [ ]:
Terminal.cashflow_plot(cash_flows)

In [ ]:
Terminal.cashflow_plot(cash_flows_WACC_real)

## SandBox

In [ ]:
Terminal.NPV()